# Generate features

## Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Get the parent directory of the current notebook
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../src"))

# Add the parent directory to sys.path
sys.path.insert(0, parent_dir)


from generate_features import (
    get_NDVI,
    get_SAVI,
    get_NDMI,
    get_NDWI,
    get_NDBI,
    get_NDSI,
    get_BSI,
    get_sobel,
    computeGLCM,
    get_NDLI,
    get_NDVI705,
    get_GNDVI,
    get_EVI2,
    get_GLI,
    get_NDYI)

from scripting._process_dems import load_dems, calculate_aspect_from_dems, calculate_slope_from_dems
from scripting._process_composites import load_composites

import rasterio
import rioxarray
import dask.distributed
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import yaml

def read_yaml(file_path: str) -> dict:
    with open(file_path, 'r') as yaml_file: return yaml.safe_load(yaml_file)

def fix_paths_for_nb(input_dict, old_substring = "/home/hrlcuser/media", new_substring = "/media/datapart/lucazanolo"):
    return {
        key: (value.replace(old_substring, new_substring) if isinstance(value, str) else value)
        for key, value in input_dict.items()
    }


## Parameters

In [ ]:
parameters = fix_paths_for_nb(read_yaml("/home/lucazanolo/luca-zanolo/scripts/config_files/4.generate_features.yaml"))


# Output
os.makedirs(parameters["output_path"], exist_ok=True)
features_date = parameters["features_date"]
features_folder_path = f"{parameters['output_path']}/{str(features_date)[:7]}"         
os.makedirs(features_folder_path, exist_ok=True)

SAVE_DATA = False
parameters

## Load composites, DEMs, slope and aspect

In [ ]:
with dask.distributed.Client(
    processes=False,
    threads_per_worker=(os.cpu_count() or 2),
) as client:
    print(f"Dask dashboard: {client.dashboard_link}")

    features_year, features_month = parameters['features_date'].split("-")
    if len(features_month) > 1 and features_month[0] == '0':
        features_month = features_month[1]

    print("Loading composites.")
    composites = load_composites(parameters['composites_path'], year=features_year, tile=parameters['tile_id'], month = features_month)

    dems = load_dems(parameters['dems_path'],year=parameters['dems_year'], tile=parameters['tile_id'])
    slope = calculate_slope_from_dems(dems.band_data)
    aspect = calculate_aspect_from_dems(dems.band_data)
    
    composites = composites.assign({"dems":dems.band_data,
                        "slopes":slope,
                        "aspects":aspect}).unify_chunks()
    
    # Fixed GLCM calculation with first month of the year and band 4
    band_data = composites.sel(band = parameters['band_to_use'], tile = parameters['tile_id']).band_data.squeeze() # Shape [time, 10980, 10980]
    
    print(f"Dems:\n{dems}\n\n")
    print(f"Aspect:\n{aspect}\n\n")
    print(f"Slope:\n{slope}\n\n")
    print(f"Composites:\n{composites}\n\n")
    print(f"Composites band data:\n{band_data}\n\n")



## Generate GLCM features

In [ ]:
with dask.distributed.Client(
    processes=False,
    threads_per_worker=(os.cpu_count() or 2),
) as client:
    print(f"Dask dashboard: {client.dashboard_link}")

    all_features = []
    all_glcms = []

    print(f"Generating GLCM and related features from time: {features_date} ...")
                                        
    glcm_features_dict = computeGLCM(band_data.values, parameters['window_size'], parameters['glcm_distance'], parameters['glcm_angle'], parameters['batch_size'])

    glcm_features = {
        feature_name: xr.DataArray(
            glcm_features_dict[feature_name][np.newaxis, :, :],  # Add a new axis for the time dimension
            dims=["time", "y", "x"],
            coords={
                "y": band_data.coords['y'],
                "x": band_data.coords['x'],
                "time": band_data.coords['time'],  # Use the time coordinates from the original data
                "spatial_ref": band_data.spatial_ref,
            },
            name=feature_name,
        ).astype(np.float32)
        for feature_name in glcm_features_dict.keys()
    }

### Save GLCM Features

In [ ]:
print(f"Saving features ...")   

for f_name, f_dt in glcm_features.items():   
    
    f_path = f"{features_folder_path}/GLCM{f_name}_{str(features_date)[:7]}.tif"
    print(f"Saving {f_name} feature to {f_path}")

    if not os.path.exists(f_path):  
        f_dt.rio.to_raster(f_path, compress="DEFLATE", num_threads="all_cpus")
        print(f"Features saved at {f_path}")
    else:
        print(f"Features {f_name} at {features_date} already exists. Remove it to recompute.")


 ## Other features

In [ ]:

with dask.distributed.Client(
    processes=False,
    threads_per_worker=(os.cpu_count() or 2),
) as client:
    print(f"Dask dashboard: {client.dashboard_link}")

    features_funcs = {
        "NDVI" : get_NDVI,
        "NDVI705" : get_NDVI705,
        "GNDVI" : get_GNDVI,
        "NDYI" : get_NDYI,
        "EVI2" : get_EVI2,
        "SAVI" : get_SAVI,
        "NDMI" : get_NDMI,
        "NDWI" : get_NDWI,
        "NDBI" : get_NDBI,
        "NDSI" : get_NDSI,
        "BSI" : get_BSI,
        "NDLI" : get_NDLI,
        "Sobel" : get_sobel,
        "GLI" : get_GLI
    }
    
    all_features = {}
    all_features_values = {}
    composites.band_data.compute()
    
    for feature_name, feature_func in features_funcs.items():
        print(f"\nProcessing feature {feature_name}")
        
        feature_path = f"{features_folder_path}/{feature_name}_{str(features_date)[:7]}.tif"
        print(f"Path: {feature_path}")

        if not os.path.exists(feature_path):
            print("Computing ...")
            
            if feature_name == "Sobel":
                feature = feature_func(all_features['NDVI'])
            else:
                feature = feature_func(composites.band_data)
            feature.rio.to_raster(feature_path, compress="DEFLATE", num_threads="all_cpus")
        else:
            print("Loading from disk ...")
            feature = rioxarray.open_rasterio(feature_path)
        
        all_features[feature_name] = feature
        all_features_values[feature_name] = feature.values
        
        

## Composite and Features info

### Show composite bands values info

In [ ]:

def band_preparation(band):
   return (band / 10000).clip(0,1)

for band in composites.band.values:
    b = composites.band_data.sel(band = band).values.squeeze()
    print(f"Band {band}:")
    print(f"Before preparation:")
    print(f"  Min: {b.min()} | Max: {b.max()}")
    print(f"  Std: {b.std()} | Mean: {b.mean()} | Var: {b.var()}")
    print(f"  NaN count: {np.sum(np.isnan(b))}")
    b = band_preparation(b)
    print(f"After preparation:")
    print(f"  Min: {b.min()} | Max: {b.max()}")
    print(f"  Std: {b.std()} | Mean: {b.mean()} | Var: {b.var()}")
    print(f"  NaN count: {np.sum(np.isnan(b))}\n")


In [ ]:
for name, feat in all_features_values.items():
    
    print(f"{name} - Mean: {np.mean(feat):.2f} - Std: {np.std(feat):.2f} - {name} - Min: {np.min(feat):.2f} - Max: {np.max(feat):.2f}")


### Plot features with info

In [ ]:
for name, feat in all_features_values.items():
    
    if name not in ['Sobel']: continue
    print(f"{name} - Mean: {np.mean(feat):.2f} - Std: {np.std(feat):.2f}")

    plt.figure(figsize=(40, 40)) 
    plt.imshow(feat.squeeze())  # Use an appropriate colormap
    plt.colorbar()  # Add colorbar for reference
    plt.title(f"{name} - Mean: {np.mean(feat):.2f} - Std: {np.std(feat):.2f}")
    plt.show()


## Generate reports

In [ ]:
def create_feature_extraction_report(parameters, id, save_path="features_report.png", show_plot=True):
    
    feature_files = [
        f"GLCMASM_{id}.tif",
        f"GLCMcontrast_{id}.tif",
        f"GLCMcorrelation_{id}.tif",
        f"GLCMdissimilarity_{id}.tif",
        f"GLCMenergy_{id}.tif",
        f"GLCMhomogeneity_{id}.tif",
        f"NDVI_{id}.tif",
        f"NDVI705_{id}.tif",
        f"GNDVI_{id}.tif",
        f"NDYI_{id}.tif",
        f"EVI2_{id}.tif",
        f"SAVI_{id}.tif",
        f"NDMI_{id}.tif",
        f"NDWI_{id}.tif",
        f"NDBI_{id}.tif",
        f"NDSI_{id}.tif",
        f"BSI_{id}.tif",
        f"NDLI_{id}.tif",
        f"Sobel_{id}.tif",
        f"GLI_{id}.tif"
    ]

    # Titles for the subplots with new features added
    feature_titles = [
        "GLCM ASM",
        "GLCM Contrast",
        "GLCM Correlation",
        "GLCM Dissimilarity",
        "GLCM Energy",
        "GLCM Homogeneity",
        "NDVI",
        "NDVI705",
        "GNDVI",
        "NDYI",
        "EVI2",
        "SAVI",
        "NDMI",
        "NDWI",
        "NDBI",
        "NDSI",
        "BSI",
        "NDLI",
        "Sobel",
        "GLI"
    ]
    
    num_features = len(feature_files)
    rows = (num_features // 4) + (1 if num_features % 4 != 0 else 0)
    fig, axes = plt.subplots(rows, 4, figsize=(20, rows * 5))
    axes = axes.flatten()

    for i, (feature_file, title) in enumerate(zip(feature_files, feature_titles)):
        file_path = os.path.join(parameters['output_path'], report_id, feature_file)
        try:
            with rasterio.open(file_path) as src:
                # Downsample the image to reduce resolution
                data = src.read(
                    1,
                    out_shape=(
                        1,
                        src.height,
                        src.width
                    ))
                im = axes[i].imshow(data, cmap='viridis')
                fig.colorbar(im, ax=axes[i], shrink=0.7)
                axes[i].set_title(f"{title} - Mean: {np.mean(data):.2f} - Std: {np.std(data):.2f}")
                axes[i].axis("off")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            axes[i].axis("off")
            axes[i].set_title(f"{title}\n(Missing)")

    for ax in axes[num_features:]:
        ax.axis("off")
    
    fig.savefig(save_path)
    if show_plot:
        plt.show()
    plt.close(fig)

    print(f"Feature extraction report saved at {save_path}")

output_path = parameters["output_path"]
report_id = "2019-01" # Plot this folder of features
features_folder_path = f"{output_path}/{report_id}"
report_path = f"{output_path}/reports"
os.makedirs(report_path, exist_ok=True)
report_path = f"{report_path}/{report_id}.png"


create_feature_extraction_report(
    parameters=parameters,
    id=report_id,
    save_path=report_path,
    show_plot=True
)
